In [69]:
# Imports
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
import  matplotlib.pyplot as plt

In [66]:
data = fetch_olivetti_faces()
images = data.images.reshape((-1, 1, 64, 64))
labels = np.arange(len(images))
images_tensor = torch.tensor(images, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)
dataset = torch.utils.data.TensorDataset(images_tensor, labels_tensor)

index_to_split = int(0.2 * len(images))
test_set = torch.utils.data.TensorDataset(images_tensor[index_to_split:], images_tensor[index_to_split:])

train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [65]:
# Display a picture of dataset
def show_picture(picture_index: int):
    image = data.images[picture_index]
    plt.imshow(image, cmap='gray')
    plt.axis('off')
    plt.show()

In [72]:
# Each picture of the dataset is square 64px x 64px
picture_size = 64

class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(picture_size * picture_size, picture_size * picture_size)
        self.fc2 = nn.Linear(picture_size * picture_size, picture_size * picture_size)
        self.fc3 = nn.Linear(picture_size * picture_size, picture_size * picture_size)
        self.fc4 = nn.Linear(picture_size * picture_size, picture_size * picture_size)

    def forward(self, x):
        # Forward pass through the layers with ReLU activation
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [74]:
EPOCH = 2

model = CNN_model()
loss_fonct = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(EPOCH): #training
    for batch in train_loader:
        optimizer.zero_grad()
        images, labels = batch
        output = model.forward(images.reshape(images.shape[0], 64 * 64))
        loss = loss_fonct(output, labels)
        loss.backward()
        optimizer.step()

total, correct = 0, 0
for image, label in test_set:
    output = model.forward(image.reshape(1, 64 * 64))
    # if (output.argmax(dim=1).item() == label):
    #     correct += 1
    total += 1

print("Accuracy: ", str(correct / total * 100) + "%")

Accuracy:  0.0%
